In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import os
import sys
import argparse
import json
import pandas as pd
import keras
import tensorflow as tf
assert(tf.__version__ == "1.15.2")
assert(keras.__version__ == "2.2.4")

# Define working directory
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)
ROOT_DIR = os.path.abspath("../../")
sys.path.append(ROOT_DIR)

from mrcnn import utils
import mrcnn.model as modellib
from utils.dataset import BloodCellDataset
from utils.configs import BloodCellConfig

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "maskrcnn-master","logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "maskrcnn-master", "mask_rcnn_coco.h5")

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "BCCD_Dataset/BCCD")

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "segmentation_WBC")

if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

IMAGE_COLLECTION_PATH = r'../blood_cell_dataset.pkl'


In [8]:
if not os.path.exists(IMAGE_COLLECTION_PATH):
    raise Exception(f"IC_PATH @ {IMAGE_COLLECTION_PATH} not found.")
else:
    dataset = pd.read_pickle(IMAGE_COLLECTION_PATH)
    train_samples = dataset[dataset["role"] == "train"]
    val_samples = dataset[dataset["role"] == "val"]

print(f"Number of samples in training set: {len(train_samples)}")
print(f"Number of samples in valid set: {len(val_samples)}")

Number of samples in training set: 395
Number of samples in valid set: 69


In [ ]:
dataset_train = BloodCellDataset(IMAGE_DIR, train_sample_id)
dataset_train.load_kernel()
dataset_val = BloodCellDataset(IMAGE_DIR, val_sample_id)
dataset_val.load_kernel()

In [ ]:
len(dataset_train.image_ids), len(dataset_val.image_ids)

In [ ]:
config = BloodCellConfig()
config.STEPS_PER_EPOCH = 400
config.LEARNING_RATE = 5e-4
config.NAME = "BloodCell"
config.display()

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [ ]:
init_with = "coco"

if init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    model_path = os.path.join(MODEL_DIR, model_branch,
                              f"mask_rcnn_BloodCell_{checkpoint_epoch:04d}.h5")
    # Load the last model you trained and continue training
    model.load_weights(model_path, by_name=True)

In [ ]:
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=10,
            layers='heads')

In [ ]:
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=100,
            layers="all")